In [8]:
import csv, re
import pandas as pd

In [29]:
vcffile = '../tests/vcfs/test.vcf'

def parse_thermo_vcf(vcf, infocol=-3, formatcol=-1, formatdesccol=-2):
    '''
    Les inn vcf til csv reader:
    Fjerner samtidig nocalls
    
    Lagrer i pandas df på vei ut
    
    
    '''
    vcf = csv.reader(open(vcf,'r+'),delimiter="\t")
    
    df = pd.DataFrame()
    
    for count,var in enumerate(vcf):
        if not var[0].startswith("#") and not var[-1].startswith("./.") and not var[-1].startswith("0/0"):
        #if not var[0].startswith("#") and var[-1].startswith("0/0:175:2435:1995:2380:1995:0:0:0:0:0:1209:1171:0:0:1029:966"):
            
                #print var
                # Eksploder FORMAT til dict
                #print {var[formatdesccol].split(":")[i]: var[formatcol].split(":")[i] for i in range(len(var[formatdesccol].split(":")))}
                # Eksploder INFO til Dict:
                # Ta høyde for at det ikke er = som HS
                #print var[infocol].split(";")
            
            
            x = [i for i in [p for p in re.split(r';(?![^\(\[]*[\]\)])', var[infocol])]]
            #print dict(e.split('=') for e in x[44])
            print x[44]
    
    #
    
   





df = parse_thermo_vcf(vcffile)
df
#df = filter_nocalls(df)

FUNC=[{'origPos':'11187893','origRef':'T','normalizedRef':'T','gene':'MTOR','normalizedPos':'11187893','normalizedAlt':'C','gt':'pos','coding':'c.6034-30A>G','transcript':'NM_004958.4','function':'unknown','protein':'p.?','location':'intronic','origAlt':'C'}]
FUNC=[{'origPos':'27089638','origRef':'A','normalizedRef':'A','gene':'ARID1A','normalizedPos':'27089638','normalizedAlt':'G','polyphen':'0.0','gt':'pos','codon':'AGC','coding':'c.2594A>G','sift':'0.16','grantham':'46.0','transcript':'NM_006015.6','function':'missense','protein':'p.Asn865Ser','location':'exonic','origAlt':'G','exon':'8'}]
VARB=0.0305689
FUNC=[{'origPos':'40363054','origRef':'G','normalizedRef':'G','gene':'MYCL','normalizedPos':'40363054','normalizedAlt':'C','polyphen':'0.0','gt':'pos','codon':'AGT','coding':'c.1175C>G','sift':'1.0','grantham':'58.0','transcript':'NM_001033082.3','function':'missense','protein':'p.Thr392Ser','location':'exonic','origAlt':'C','exon':'3'}]
FUNC=[{'origPos':'65311214','origRef':'G','no

IndexError: list index out of range

In [7]:
mini = df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [5]:
mini

NameError: name 'mini' is not defined

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [102]:
[dict(i.split('=') for i in x) for x in list(mini['INFO'].str.split(";"))]


ValueError: dictionary update sequence element #44 has length 3; 2 is required

In [78]:
d = [dict(x.split('=') for i in x) for x in mini['INFO'].str.split(';')]
d
#{k: v for d in L for k, v in d.items()}
#df = pd.DataFrame.from_dict()
#pd.DataFrame.from_dict(L)
#df = df[['name']].join(pd.DataFrame(L))

AttributeError: 'list' object has no attribute 'split'

In [57]:
{k: [v] for k, v in L.items()}

AttributeError: 'list' object has no attribute 'items'

In [11]:
[dict(i.split(';') for i in x) for x in mini['INFO']]

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [ ]:
def expand_info_col(df):